In [1]:
val sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
48,application_1612876649427_0002,spark,idle,Link,Link


SparkSession available as 'spark'.
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@26306cd6


In [2]:
import org.apache.spark.sql.types.{FloatType,LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.types.{FloatType, LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window


In [3]:
import com.logicalclocks.hsfs._

import com.logicalclocks.hsfs._


In [4]:
val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@60a8bcea
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=67, name='amlsim_featurestore', projectId=119, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@12624486}


In [5]:
val df_all = fs.getFeatureGroup("simts_features", 1).read()

df_all: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [f_19: double, f_13: double ... 364 more fields]


In [6]:
df_all.count()

res1: Long = 60000


In [7]:
df_all.where($"target" === 1).count()

res2: Long = 1000


In [8]:
df_all.where($"target" === 0).count()

res3: Long = 59000


In [9]:
print(df_all.columns.toSeq)

WrappedArray(f_19, f_13, f_22, f_23, f_33, f_51, f_58, f_64, f_68, f_72, f_73, f_77, f_83, f_88, f_90, f_92, f_94, f_100, f_103, f_105, f_106, f_107, f_112, f_116, f_117, f_121, f_124, f_125, f_127, f_132, f_141, f_146, f_151, f_158, f_159, f_163, f_165, f_168, f_172, f_178, f_182, f_184, f_190, f_191, f_196, f_204, f_205, f_208, f_214, f_220, f_222, f_223, f_224, f_225, f_228, f_231, f_234, f_237, f_240, f_253, f_257, f_264, f_271, f_272, f_274, f_276, f_279, f_280, f_283, f_286, f_287, f_293, f_303, f_308, f_313, f_316, f_321, f_322, f_324, f_331, f_340, f_342, f_344, f_346, f_352, f_361, f_1, f_5, f_3, f_2, f_6, f_8, f_15, f_34, f_38, f_40, f_42, f_47, f_54, f_65, f_79, f_80, f_81, f_85, f_91, f_93, f_95, f_98, f_102, f_104, f_111, f_113, f_114, f_118, f_120, f_123, f_131, f_133, f_134, f_136, f_144, f_145, f_153, f_154, f_156, f_160, f_164, f_167, f_169, f_173, f_177, f_185, f_186, f_187, f_189, f_194, f_200, f_209, f_211, f_230, f_235, f_242, f_248, f_249, f_250, f_251, f_262, f_2

In [10]:
print(df_all.columns.slice(0,365).map(a => a.toString ).toSeq)

WrappedArray(f_19, f_13, f_22, f_23, f_33, f_51, f_58, f_64, f_68, f_72, f_73, f_77, f_83, f_88, f_90, f_92, f_94, f_100, f_103, f_105, f_106, f_107, f_112, f_116, f_117, f_121, f_124, f_125, f_127, f_132, f_141, f_146, f_151, f_158, f_159, f_163, f_165, f_168, f_172, f_178, f_182, f_184, f_190, f_191, f_196, f_204, f_205, f_208, f_214, f_220, f_222, f_223, f_224, f_225, f_228, f_231, f_234, f_237, f_240, f_253, f_257, f_264, f_271, f_272, f_274, f_276, f_279, f_280, f_283, f_286, f_287, f_293, f_303, f_308, f_313, f_316, f_321, f_322, f_324, f_331, f_340, f_342, f_344, f_346, f_352, f_361, f_1, f_5, f_3, f_2, f_6, f_8, f_15, f_34, f_38, f_40, f_42, f_47, f_54, f_65, f_79, f_80, f_81, f_85, f_91, f_93, f_95, f_98, f_102, f_104, f_111, f_113, f_114, f_118, f_120, f_123, f_131, f_133, f_134, f_136, f_144, f_145, f_153, f_154, f_156, f_160, f_164, f_167, f_169, f_173, f_177, f_185, f_186, f_187, f_189, f_194, f_200, f_209, f_211, f_230, f_235, f_242, f_248, f_249, f_250, f_251, f_262, f_2

In [11]:
import org.apache.spark.ml.feature.{MinMaxScaler,StandardScaler,Normalizer, MaxAbsScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().setInputCols(df_all.columns.slice(0,365).map(a => a.toString).toArray).setOutputCol("features")

val df_all_features = assembler.transform(df_all)

// scaler instance with the min(-1) and max(1) 
val mms = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(-1)


val mms_tr = mms.setInputCol("features").setOutputCol("mms")

val trainingFeaturesPipeline = (new Pipeline()
  .setStages(Array(mms_tr)))

val trainingFeaturesDF = trainingFeaturesPipeline.fit(df_all_features).transform(df_all_features)

import org.apache.spark.ml.feature.{MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_82f3c831cfc8
df_all_features: org.apache.spark.sql.DataFrame = [f_19: double, f_13: double ... 365 more fields]
mms: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_8b7dc1535626
mms_tr: mms.type = minMaxScal_8b7dc1535626
trainingFeaturesPipeline: org.apache.spark.ml.Pipeline = pipeline_ee1dba0c8339
trainingFeaturesDF: org.apache.spark.sql.DataFrame = [f_19: double, f_13: double ... 366 more fields]


In [12]:
import org.apache.spark.ml.linalg.DenseVector
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)

val final_df_all = trainingFeaturesDF.select($"target",$"mms").
             withColumn("target",$"target".cast(FloatType)).
             withColumn("mms",toArrUdf($"mms").cast(ArrayType(FloatType)))

import org.apache.spark.ml.linalg.DenseVector
toArr: Any => Array[Double] = <function1>
toArrUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
final_df_all: org.apache.spark.sql.DataFrame = [target: float, mms: array<float>]


In [13]:
val van = final_df_all.where($"target"===1)
val ben = final_df_all.where($"target"===0) 

van: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]
ben: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [14]:
// Now that the data has been prepared, let's split the dataset into a training and test dataframe
val Array(benTrainDF, benEvalDF) = final_df_all.randomSplit(Array(0.8, 0.02),seed = 12345)

benTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]
benEvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [15]:
benTrainDF.count()

res16: Long = 58539


In [16]:
benEvalDF.count()

res17: Long = 1461


In [17]:
val EvalDF = benEvalDF.union(van)

EvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [18]:
EvalDF.where($"target"===1).count()

res18: Long = 1024


In [19]:
EvalDF.where($"target"===0).count()

res19: Long = 1437


In [20]:
val ben_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("ben_td")
                .description("Ben simulated Time Series Training Dataset")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
ben_sim_ts_td_meta.save(benTrainDF)

ben_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@468e109


In [21]:
val eval_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("eval_td")
                .description("Simulated Time Series Training Dataset for evaluation")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
eval_sim_ts_td_meta.save(EvalDF)

eval_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@5dc93511


In [ ]:
val testDf = EvalDF.where($"target"===1)

val test_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("test_td")
                .description("Simulated Time Series Training Dataset for testing")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
test_sim_ts_td_meta.save(testDf)